# Fish Dataset Creation – Full Pipeline

## Step 1: Create calibration files cm_ground.npy and cm_vertical.npy

In [ ]:

import cv2, numpy as np

cap = cv2.VideoCapture(r"C:\Users\shain\Downloads\FISH DATASET\FISH DATASET\fish01\front view\front.mp4.mp4")
ret,frame=cap.read()
cv2.imshow("frame",frame); cv2.waitKey(0); cv2.destroyAllWindows()

points=[]
def click(event,x,y,flags,param):
    if event==cv2.EVENT_LBUTTONDOWN:
        points.append((x,y))
        print(points)

cv2.imshow("Click two ends of ONE FLOOR 10cm tape",frame)
cv2.setMouseCallback("Click two ends of ONE FLOOR 10cm tape",click)
cv2.waitKey(0); cv2.destroyAllWindows()

(x1,y1),(x2,y2)=points
px=((x2-x1)**2+(y2-y1)**2)**0.5
CM_GROUND=10/px
np.save("cm_ground.npy",CM_GROUND)
print("Saved cm_ground.npy",CM_GROUND)

points=[]
cv2.imshow("Click TOP & BOTTOM of ONE WALL 10cm tape",frame)
cv2.setMouseCallback("Click TOP & BOTTOM of ONE WALL 10cm tape",click)
cv2.waitKey(0); cv2.destroyAllWindows()

(x1,y1),(x2,y2)=points
px=abs(y2-y1)
CM_VERTICAL=10/px
np.save("cm_vertical.npy",CM_VERTICAL)
print("Saved cm_vertical.npy",CM_VERTICAL)


## Step 2: Load calibration

In [ ]:

import numpy as np
CM_GROUND=np.load("cm_ground.npy")
CM_VERTICAL=np.load("cm_vertical.npy")
print(CM_GROUND,CM_VERTICAL)


## Step 3: Extraction functions

In [ ]:

import cv2

def top_clear(frame):
    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    m=cv2.inRange(hsv,(0,40,30),(180,255,255))
    return cv2.countNonZero(m)>8000

def front_clear(frame):
    g=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    e=cv2.Canny(g,40,120)
    return e.sum()>30000

def top_measure(frame,CM):
    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    m=cv2.inRange(hsv,(0,40,30),(180,255,255))
    cnt,_=cv2.findContours(m,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    c=max(cnt,key=cv2.contourArea)
    area=cv2.contourArea(c)*CM**2
    peri=cv2.arcLength(c,True)*CM
    rect=cv2.minAreaRect(c)
    L,W=max(rect[1])*CM,min(rect[1])*CM
    return L,W,area,peri

def front_height(frame,CM):
    g=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    e=cv2.Canny(g,40,120)
    ys,_=np.where(e>0)
    return (ys.max()-ys.min())*CM


## Step 4: Process one fish

In [ ]:

def process_fish(folder,fish_id,weight):
    top=cv2.VideoCapture(folder+'/top.mp4')
    front=cv2.VideoCapture(folder+'/front.mp4')
    feats=[]
    while len(feats)<25:
        r1,f1=top.read(); r2,f2=front.read()
        if not r1 or not r2: break
        if top_clear(f1) and front_clear(f2):
            L,W,A,P=top_measure(f1,CM_GROUND)
            H=front_height(f2,CM_VERTICAL)
            feats.append([L,W,H,A,P])
    return [fish_id,weight,*np.mean(feats,axis=0)]


## Step 5: Build dataset

In [ ]:

import pandas as pd

rows=[]
# rows.append(process_fish('fish01','Fish1',67.49))
df=pd.DataFrame(rows,columns=['FishID','Weight','Length','Width','Height','Area','Perimeter'])
df.to_csv('fish_dataset.csv',index=False)
df
